# Abalation Testing and Inference Mode Flow Forecast
In this tutorial we will use Flow Forecast in order to run an abalation test on some of our river flow data. We will continue studying the flow of the Virgin River though this could of course work for any data... S
D

In [ ]:
from google.colab import auth
from datetime import datetime
import os
auth.authenticate_user()
!git clone https://github.com/AIStream-Peelout/flow-forecast.git
os.environ['MODEL_BUCKET'] = "predict_cfs"
os.environ["ENVIRONMENT_GCP"] = "Colab"
os.environ["GCP_PROJECT"] = "gmap-997"


Cloning into 'flow-forecast'...
remote: Enumerating objects: 13631, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (324/324), done.
remote: Total 13631 (delta 328), reused 292 (delta 163), pack-reused 13143
Receiving objects: 100% (13631/13631), 4.01 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (9879/9879), done.


In [ ]:
import os
os.chdir('flow-forecast')
!pip install -r  requirements.txt
!python setup.py develop
!mkdir data
#!wandb login

In [ ]:
import pandas as pd
!gsutil cp gs://aistream-datasets/flowdb/09405500AZC_flow.csv .
df = pd.read_csv("09405500AZC_flow.csv")
df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("09405500AZC_flow.cs")

Copying gs://aistream-datasets/flowdb/09405500AZC_flow.csv...
- [1 files][ 19.6 MiB/ 19.6 MiB]                                                
Operation completed over 1 objects/19.6 MiB.                                     


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,7,8,9,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from flood_forecast.deployment.inference import InferenceMode
import json
!gsutil cp gs://predict_cfs/experiments/20_April_202110_41PM.json config.json
new_water_data_path = "09405500AZC_flow.csv"
weight_path = "gs://predict_cfs/experiments/20_April_202110_41PM_model.pth"
# To do add support for loading config file from path (local or GCS)
with open("config.json") as y:
  config_test = json.load(y)
  #config_test["inference_params"]["dataset_params"]["relevant_cols"] = ["cfs1", 'precip', "temp"]
d = InferenceMode(336, 30, config_test, new_water_data_path, weight_path, "river")


Copying gs://predict_cfs/experiments/20_April_202110_41PM.json...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
gs://predict_cfs/experiments/20_April_202110_41PM_model.pth
Blob experiments/20_April_202110_41PM_model.pth downloaded to data/predict_cfs/experiments/20_April_202110_41PM_model.pth.
Weights sucessfully loaded
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading 09405500AZC_flow.cs
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading 09405500AZC_flow.cs
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now l

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Plotting a new datetime

In [ ]:
d.make_plots(datetime(2020, 9, 20), new_water_data_path, wandb_plot_id="cool_plot" )

In [ ]:
d.make_plots(datetime(2020, 3, 13), new_water_data_path, wandb_plot_id="cool_plot2" )

### Testing Extreme Values

In [ ]:
# We will now grab a random slice of data (don't worry we are going to change this)
data = d.model.training[2]

In [ ]:
sample1 = data[0][:, :]

In [ ]:


sample1.shape

torch.Size([24, 8])

Above we have slice of data from the train loader. Since our original list of relevant columns was `["cfs","tmpf","p01m","dwpf"]` That means index 2 will be precipitation. Lets replace this index with a really large value

In [ ]:
sample1[:, 2]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
sample1[22, 2] = 22
sample1[22, 2] = 25
sample1[23, 2] = 25 

In [ ]:
sample1[:, 2]

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25., 25.])

In [ ]:
d.model.model(data[0][:, :].unsqueeze(0))

/content/flow-forecast/flood_forecast/transformer_xl/transformer_bottleneck.py:261: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([[-0.1548,  0.0775, -0.1576, -0.1607, -0.1500]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
d.model.model(sample1.unsqueeze(0))

/content/flow-forecast/flood_forecast/transformer_xl/transformer_bottleneck.py:261: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([[-0.1548,  0.0775, -0.1576, -0.1607, -0.1500]],
       grad_fn=<SqueezeBackward1>)

 From this we can see that the model is unfortunately not looking at precipitation at all while forecasting... FUCK FUCK FUCK

In [ ]:
d.model.model(sample1.unsqueeze(0))

/content/flow-forecast/flood_forecast/transformer_xl/transformer_bottleneck.py:261: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([[-0.2228,  0.0929, -0.1725, -0.1528, -0.1757]],
       grad_fn=<SqueezeBackward1>)